In [1]:
import tensorflow as tf
import numpy as np
import glob
import json
from collections import Counter
from tqdm import tqdm
from snownlp import SnowNLP
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split as split
from deap import base, creator, tools, algorithms
from scipy.stats import bernoulli
from bitstring import BitArray

a = tf.test.is_built_with_cuda()
b = tf.test.is_gpu_available(
    cuda_only=False,
    min_cuda_compute_capability=None
)     
print(a)
print(b)
np.random.seed(1000)

True
True


In [2]:
def split_dataset():
    paths = glob.glob('./dataset/json/poet.*.json')
    poets = []
    for path in paths:
        data = open(path, 'r', encoding='utf-8').read()
        data = json.loads(data)
        #print(data)
        for item in data:
            content = ''.join(item['paragraphs'])
            if len(content)>= 24 and len(content) <= 32:
                content = SnowNLP(content)
                content=content.han
                if len(content)%4==0:
                    poets.append('[' + content + ']')
    print('We have %d Chinese ancient poets' % len(poets), poets[0], poets[-1])
    training_data, validation_data = split(poets, test_size = 0.20, random_state = 1000)

    return training_data, validation_data


In [3]:
training_data, validation_data = split_dataset()

We have 104147 Chinese ancient poets [欲出未出光辣达，千山万山如火发。须臾走向天上来，逐却残星赶却月。] [书劒催人不暂闲，洛阳羁旅复秦关。容颜岁岁愁边改，乡国时时梦里还。]


In [4]:
print(len(training_data))

83317


In [5]:
def prepocess_data(dataset):
    chars = []
    for item in dataset:
        chars += [c for c in item]
    print('We have %d words' % len(chars))

    chars = sorted(Counter(chars).items(), key=lambda x:x[1], reverse=True)
    print('We have %d different words in dataset' % len(chars))
    chars = [c[0] for c in chars]
    char2id = {c: i + 1 for i, c in enumerate(chars)}
    id2char = {i + 1: c for i, c in enumerate(chars)}
    
    return char2id, id2char


In [6]:
def process_data(char2id, id2char, dataset, ga_individual_solution):   
# Decode GA solution to integer for batch_size and hidden_size
#     batch_size_bits = BitArray(ga_individual_solution[0:8])
#     batch_size = batch_size_bits.uint
#     print('\nBatch Size: ', batch_size)
    batch_size = 512
    X_data = []
    Y_data = []

    for b in range(len(dataset) // batch_size):
        start = b * batch_size
        end = b * batch_size + batch_size
        batch = [[char2id[c] for c in dataset[i]] for i in range(start, end)]
        maxlen = max(map(len, batch))
        X_batch = np.full((batch_size, maxlen - 1), 0, np.int32)
        Y_batch = np.full((batch_size, maxlen - 1), 0, np.int32)
        for i in range(batch_size):
            X_batch[i, :len(batch[i]) - 1] = batch[i][:-1]
            Y_batch[i, :len(batch[i]) - 1] = batch[i][1:]
        X_data.append(X_batch)
        Y_data.append(Y_batch)
    return X_data, Y_data
    
    

In [7]:
def build_model_and_train_evaluate(ga_individual_solution):
    #training_data, validation_data = split_dataset()
    training_char2id, training_id2char = prepocess_data(training_data)
    validation_char2id, validation_id2char = prepocess_data(validation_data)
    
    import pickle
    with open('dictionary.pkl', 'wb') as fw:
        pickle.dump([training_char2id, training_id2char], fw)
    
    X_training_data, Y_training_data = process_data(training_char2id, training_id2char, training_data, ga_individual_solution)
    X_validation_data, Y_validation_data = process_data(validation_char2id, validation_id2char, validation_data, ga_individual_solution)
    
    tf.reset_default_graph()
    batch_size = 512
    num_layer = 2
    embedding_size = 256
    
    hidden_size = []
    hidden_size_bits_1 = BitArray(ga_individual_solution[0:8])
    hidden_size.append(hidden_size_bits_1.uint)
    print('\nHidden Size 1: ', hidden_size[0])
    hidden_size_bits_2 = BitArray(ga_individual_solution[8:])
    hidden_size.append(hidden_size_bits_2.uint)
    print('\nHidden Size 2: ', hidden_size[1])
    
    
    X_training= tf.placeholder(tf.int32, [batch_size, None])
    Y_training= tf.placeholder(tf.int32, [batch_size, None])
    learning_rate = tf.Variable(0.0, trainable=False)
    ikeep_prob = tf.placeholder(tf.float32, name='ikeep_prob')
    okeep_prob = tf.placeholder(tf.float32, name='okeep_prob')
    cell = tf.nn.rnn_cell.MultiRNNCell(
        [tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(hidden_size[i]), input_keep_prob=ikeep_prob, output_keep_prob=ikeep_prob )for i in range(num_layer)], 
        state_is_tuple=True)

    initial_state = cell.zero_state(batch_size, tf.float32)
    tr_embeddings = tf.Variable(tf.random_uniform([len(training_char2id) + 1, embedding_size], -1.0, 1.0))
    tr_embedded = tf.nn.embedding_lookup(tr_embeddings, X_training)
    tr_outputs, tr_last_states = tf.nn.dynamic_rnn(cell, tr_embedded, initial_state=initial_state)
    tr_outputs = tf.reshape(tr_outputs, [-1, hidden_size[1]])                # batch_size * max_time, hidden_size

    training_logits = tf.layers.dense(tr_outputs, units=len(training_char2id) + 1)       # batch_size * max_time, len(char2id) + 1
    training_logits = tf.reshape(training_logits, [batch_size, -1, len(training_char2id) + 1]) # batch_size, max_time, len(char2id) + 1
    probs = tf.nn.softmax(training_logits)                                   # batch_size, max_time, len(char2id) + 1
    loss = tf.reduce_mean(tf.contrib.seq2seq.sequence_loss(training_logits, Y_training, tf.ones_like(Y_training, dtype=tf.float32)))
    params = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, params), 5)
    optimizer = tf.train.AdamOptimizer(learning_rate).apply_gradients(zip(grads, params))
    
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    Loss=[]
    Validation_loss=[]
    for epoch in range(40):
        sess.run(tf.assign(learning_rate, 0.0004 * (0.97 ** epoch)))

        data_index = np.arange(len(X_training_data))
        np.random.shuffle(data_index)
        X_training_data = [X_training_data[i] for i in data_index]
        Y_training_data = [Y_training_data[i] for i in data_index]

        losses = []
        validates=[]

        for i in tqdm(range(len(X_training_data))):
            ls_,  _ = sess.run([loss, optimizer], feed_dict={X_training: X_training_data[i], Y_training: Y_training_data[i], ikeep_prob: 0.8, okeep_prob: 0.8})
            losses.append(ls_)
        for i in tqdm(range(len(X_validation_data))):
            validate_acc = sess.run([loss], feed_dict={X_training: X_validation_data[i], Y_training: Y_validation_data[i], ikeep_prob: 1, okeep_prob: 1})
            validates.append(validate_acc)
            
        
        Loss.append(np.mean(losses))
        Validation_loss.append(np.mean(validates))
    #save model
    saver = tf.train.Saver()
    saver.save(sess, './50/50_epoch')
        
    #Now the model has been trained and we want it to generate a poem
    with open('dictionary.pkl', 'rb') as fr:
        [char2id, id2char] = pickle.load(fr)
    
    tf.reset_default_graph()
    test_batch_size = 1

    X = tf.placeholder(tf.int32, [test_batch_size, None])
    Y = tf.placeholder(tf.int32, [test_batch_size, None])
    learning_rate = tf.Variable(0.0, trainable=False)

    cell = tf.nn.rnn_cell.MultiRNNCell(
        [tf.nn.rnn_cell.LSTMCell(hidden_size[i], state_is_tuple=True) for i in range(num_layer)], 
        state_is_tuple=True)
    initial_state = cell.zero_state(test_batch_size, tf.float32)

    embeddings = tf.Variable(tf.random_uniform([len(char2id) + 1, embedding_size], -1.0, 1.0))
    embedded = tf.nn.embedding_lookup(embeddings, X)

    outputs, last_states = tf.nn.dynamic_rnn(cell, embedded, initial_state=initial_state)

    outputs = tf.reshape(outputs, [-1, hidden_size[1]])
    logits = tf.layers.dense(outputs, units=len(char2id) + 1)
    probs = tf.nn.softmax(logits)
    targets = tf.reshape(Y, [-1])

    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=targets))
    params = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, params), 5)
    optimizer = tf.train.AdamOptimizer(learning_rate).apply_gradients(zip(grads, params))

    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    saver.restore(sess, tf.train.latest_checkpoint('./50'))
    
    def poetry_generate_with_topic(head):
        states_ = sess.run(initial_state)
        gen = ''
        c = '['
        i = 0
        while c != ']':
            gen += c
            x = np.zeros((test_batch_size, 1))
            x[:, 0] = char2id[c]
            probs_, states_ = sess.run([probs, last_states], feed_dict={X: x, initial_state: states_})
            probs_ = np.squeeze(probs_)
            pos = int(np.searchsorted(np.cumsum(probs_), np.random.rand() * np.sum(probs_)))
            while pos not in id2char:
                pos = int(np.searchsorted(np.cumsum(probs_), np.random.rand() * np.sum(probs_)))
            
            c = id2char[pos]
            if i<len(head):
                c=head[i]
                i+=1
                
        return gen[1:]
    
    print('Validation_loss = ', Validation_loss[-1])
    print('Loss = ', Loss[-1])
    print(poetry_generate_with_topic('边塞'))
    return Validation_loss[-1], Loss[-1]


In [8]:
population_size = 8
num_generations = 8
gene_length = 16

# As we are trying to minimize the Loss and the Validation_loss, that's why using -1.0. 
# In case, when you want to maximize accuracy for instance, use 1.0
creator.create('FitnessMulti', base.Fitness, weights = (-1.0, -1.0))
creator.create('Individual', list , fitness = creator.FitnessMulti)

toolbox = base.Toolbox()
toolbox.register('binary', bernoulli.rvs, 0.5)
toolbox.register('individual', tools.initRepeat, creator.Individual, toolbox.binary, n = gene_length)
toolbox.register('population', tools.initRepeat, list , toolbox.individual)

toolbox.register('mate', tools.cxOrdered)
toolbox.register('mutate', tools.mutShuffleIndexes, indpb = 0.6)
toolbox.register('select', tools.selRoulette)
toolbox.register('evaluate', build_model_and_train_evaluate)

population = toolbox.population(n = population_size)
r = algorithms.eaSimple(population, toolbox, cxpb = 0.4, mutpb = 0.1, ngen = num_generations, verbose = False)

We have 2713538 words
We have 7754 different words in dataset
We have 678172 words
We have 6364 different words in dataset

Hidden Size 1:  168

Hidden Size 2:  82


100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [00:04<00:00,  8.60it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [00:04<00:00,  8.60it/s]


INFO:tensorflow:Restoring parameters from ./50\50_epoch
Validation_loss =  6.1360245
Loss =  5.137557
边塞儿藏树手，凿峰万尽起楼。闭星楼外正携，只烟百湖罢长。
We have 2713538 words
We have 7754 different words in dataset
We have 678172 words
We have 6364 different words in dataset

Hidden Size 1:  229

Hidden Size 2:  63


100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [00:04<00:00,  8.51it/s]


...

100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [00:04<00:00,  9.51it/s]


INFO:tensorflow:Restoring parameters from ./50\50_epoch
Validation_loss =  6.0856853
Loss =  6.100179
边塞龙梢八文，壁野醒寒饮了
We have 2713538 words
We have 7754 different words in dataset
We have 678172 words
We have 6364 different words in dataset

Hidden Size 1:  25

Hidden Size 2:  38


100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [00:03<00:00, 10.26it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [00:03<00:00, 10.26it/s]


INFO:tensorflow:Restoring parameters from ./50\50_epoch
Validation_loss =  5.8019795
Loss =  5.6364927
边塞一陵尘雨，东春南无纷行。欲主犹君□食后，难得乐亲浮枝草。


In [9]:
best_individuals = tools.selBest(population,k = 1)
best_hidden_size_1 = None
best_hidden_size_2 = None

for bi in best_individuals:
    best_hiden_size_1_bits = BitArray(bi[0:8])
    best_hidden_size_2_bits = BitArray(bi[8:]) 
    best_hidden_size_1 = best_hiden_size_1_bits.uint
    best_hidden_size_2 = best_hidden_size_2_bits.uint
    print('\nHidden Size 1 : ', best_hidden_size_1, ', Hidden Size 2: ', best_hidden_size_2)


Hidden Size 1 :  25 , Hidden Size 2:  38
